# <font color='blue'> Starting the environment  </font>
Once *QMWORKS*  has been installed the user should run the following command to initialize the environment:

To leave the environment the following command is used

# <font color='blue'> What is QMworks?</font>
QMworks is a python library that enables executing complicated workflows of interdependent quantum chemical (QM) calculations in python. It aims at providing a common interface to multiple QM packages, enabling easy and systematic generation of the calculation inputs, as well as facilitating automatic analysis of the results. Furthermore it is build on top of the powerful Noodles framework for executing the calculations in parallel where possible.

# <font color='blue'> The basics: calling packages</font> 
Currently **QMWORKS** offers an interface with the following simulation software:
* #### SCM (ADF and DTFB)
* #### CP2K
* #### ORCA
* #### GAMESS-US
* #### DIRAC

With ``qmworks`` you can write a python script that simply calls one of the package objects 
*adf, dftb, cp2k, orca, gamess* or *dirac*.
As arguments to the call, you need to provide a ``settings`` objects defining the input of a calculation, a molecular geometry, and, optionally, a job name that enables you to find back the "raw" data of the calculation later on.

Let's see how this works:

First we define a molecule, for example by reading one from an xyz file:

In [1]:
from plams import Molecule
acetonitrile = Molecule("files/acetonitrile.xyz")
print(acetonitrile)

  Atoms: 
    1         C      2.419290      0.606560      0.000000 
    2         C      1.671470      1.829570      0.000000 
    3         N      1.065290      2.809960      0.000000 
    4         H      2.000000      0.000000      1.000000 
    5         H      2.000000      0.000000     -1.000000 
    6         H      3.600000      0.800000      0.000000 



Then we can perform geometry optimization on the molecule by a call to the dftb package object:

In [2]:
from qmworks import dftb, templates, run
job = dftb(templates.geometry, acetonitrile)
print(job)

/home/lars/anaconda3/envs/rdkit-env/lib/python3.5/site-packages/rdkit/Chem/PeriodicTable.py:131: FutureWarning: split() requires a non-empty pattern match.
  splitLine = blankExpr.split(line)


As you can see, "job" is a so-called "promised object". It means it first needs to be "run" by the Noodles scheduler to return a normal python object.

In [3]:
result = run(job)
print(result)

[09:03:58] PLAMS working folder: /home/lars/workspace/qmworks/jupyterNotebooks/plams.4050
╭─(running jobs)
│ Running dftb ...
╰(✔─)(success)


We can easily retrieve the calculated properties from the *ADF* calculation such as the dipole or the optimized geometry for use in subsequent calculations.

In [4]:
print("Dipole: ", result.dipole)
optimized_mol_adf = result.molecule
print(optimized_mol_adf)

Dipole:  [1.0864213029, -1.9278296041, -0.0]
  Atoms: 
    1         C      2.366998      0.579794      0.000000 
    2         C      1.660642      1.834189      0.000000 
    3         N      1.089031      2.847969      0.000000 
    4         H      2.100157      0.010030      0.887206 
    5         H      2.100157      0.010030     -0.887206 
    6         H      3.439065      0.764079     -0.000000 



# <font color='blue'> Settings </font> 
In the above example ``templates.geometry`` was actually a predefined Settings object.
You can define and manipulate Settings in a completely flexible manner as will be explained in this section. To facilitate combining different packages in one script, QMworks defines a set of commonly used generic keywords, which can be combined with package specific keywords, to provide maximum flexibility.

In [5]:
from qmworks import Settings
s = Settings()
s.basis = "DZP"
s.specific.adf.basis.core = "large"
s.freeze = [1,2,3]
print(s)

basis: 	DZP
freeze: 	[1, 2, 3]
specific: 	
         adf: 	
             basis: 	
                   core: 	large



This code snippet illustrates that the ``Settings`` can be specified in two ways, using generic or specific keywords. Generic keywords represent input properties that are present in most simulation packages like a *basis set* while *specific* keywords allow the user to apply specific keywords for a package that are not in a generic dictionary. Upon calling a package with such Settings object, the generic keywords are first translated into package specific keywords and combined with the relevant user defined specific keywords. Note that in the above example, the keywords after ``specific.adf`` will be used only when passed to the package ``adf`` and will be ignored if this settings object is passed to another package. Once all package specific keywords are obtained they are translated into a package specific input file based on the [PLAMS software](https://www.scm.com/doc/plams/index.html).

In QMworks/PLAMS multiple settings objects can be combined using e.g. the ``overlay`` function.

In [6]:
merged_settings = templates.geometry.overlay(s)

The *overlay* method takes as input a template containing a default set for different packages and also takes the arguments provided by the user, as shown schematically 
<img src="files/merged.png">

This overlay method merged the defaults for a given packages (*ADF* in this case) with the input supplied by the user, always given preference to the user input
<img src="files/result_merged.png" width="700">

# <font color='blue'> Combining multiple jobs </font>


Multiple jobs can be combined, while calling the run function only once. The script below combines components outlined above:

In [7]:
from plams import Molecule
from qmworks import dftb, adf, templates, run, Settings

acetonitrile = Molecule("files/acetonitrile.xyz")

dftb_opt = dftb(templates.geometry, acetonitrile, job_name="dftb_opt")

s = Settings()
s.basis = "DZP"
s.specific.adf.basis.core = "large"
print(dftb_opt.molecule)
adf_single = adf(templates.singlepoint.overlay(s), dftb_opt.molecule, job_name="adf_single")

adf_result = run(adf_single)
print(dftb_opt.molecule)

print(adf_result.energy)

╭─(running jobs)
│ Running dftb dftb_opt...
(✔)│ Running adf adf_single...
(✔)╰─(success)
-1.409487473452905


In this case the second task adf_single reads the molecule optimized in the first job dftb_opt. Note that dftb_opt as well as dftb_opt.molecule are promised objects. When ``run`` is applied to the adf_single job, noodles builds a graph of dependencies and makes sure all the calculations required to obtain ``adf_result`` are performed.

All data related to the calculations, i.e. input files generated by QMworks and the resulting output files generated by the QM packages are stored in folders with the job_names "dftb_opt" and "adf_single" inside a plams folder:

In [8]:
ls; echo 'files in plams folder:'; ls plams*

cache.json  examples.ipynb  files/  plams.4050/  tutorial.ipynb
files in plams folder:
adf_single  DFTBjob  dftb_opt  plams.4050.log


**Templates** are stored inside the ``qmworks`` package as JSON files. Below the defaults for single point calculations are shown.

In [7]:
print(templates.singlepoint)

specific: 	
         adf: 	
             basis: 	
                   type: 	SZ
             integration: 	
                         accint: 	4.0
             scf: 	
                 converge: 	1e-06
                 iterations: 	100
             xc: 	
                __block_replace: 	True
                lda: 	
         cp2k: 	
              force_eval: 	
                         dft: 	
                             basis_set_file_name: 	
                             mgrid: 	
                                   cutoff: 	400
                                   ngrids: 	4
                             potential_file_name: 	
                             print: 	
                                   mo: 	
                                      add_last: 	numeric
                                      each: 	
                                           qs_scf: 	0
                                      eigenvalues: 	
                                      eigenvectors: 	
                              

## A little discussion about graphs
*qmworks* is meant to be used for both workflow generation and execution,

<img src="files/simple_graph.png">

In [9]:
from qmworks import run


### <font color='green'> result = run(frequencies) </font>

Once you run the script, as you we will see in the next section, an input file for the *ADF* job is created

Running in **Cartesius** or **Bazis** through the *Slurm* resource manager can be done using and script like

The Slurm output looks like: